### Setup aus MetEngSim

In [1]:
import sys # loading commands to control/navigate within the system architecture
# Loading pandas, a library for data manipulation
from os.path import join
# import xlrd
import pandas as pd
# import lxml

# Loading numpy, a library fo manipulation of numbers
import numpy as np

# loading matplotlib, a library for visualization
import matplotlib.pyplot as plt
%matplotlib inline

from biolabsim import Host, Strain, Ecol
from biolabsim import measure_EnzymeLevel1, Help_GenomeGenerator, Strain, make_UpdateExpression, Help_StrainCharacterizer, Help_PromoterStrength, Help_Expr2Flux
from Bio.Seq import Seq
from pandas.core.frame import DataFrame
from copy import deepcopy

# import escher
# from escher import Builder

print('System ready')

System ready


In [2]:
# define new and changed functions

def add_TranscriptionFactor(df:DataFrame, RegID:str, EnzymeTarget):
    newdf = df
    
    # check if TF is already present, add columns to the default TF accordingly, no zero initiation of RegType column
    if any('TF_regulated' in s for s in WTdf.columns.values):
        TF = {
        'RctID': [RegID], 
        'Expression': [2], # noch zu ändern mit Help_PromoterStrength()?
        'Promoter':['GCCCATTGACAAGGCTCTCGCGCGAGTGTATAATTGCACG'],  # noch zu ändern (echt, random oder Fkt?)
        'ORF': ['ATGGAGATGAAGTAA'], # noch zu ändern (echt, random oder Fkt?)
        'Fluxes': [9], # noch zu ändern mit Help_FluxCalculator()?
        'Expr2Flux': [4.5], # noch zu ändern mit Help_Expr2Flux()?
        'TF_regulated': [0],
        'RegType': 'Hill_Activator'
        }
        dfTF = pd.DataFrame(TF)
        newdf = newdf.append(dfTF, ignore_index=True)
        
    else:
    # add TF row:
        Sequence = 'GCCCATTGACAAGGCTCTCGCGGCCAGGTATAATTGCACG'
        TF = {
        'RctID': [RegID], 
        'Expression': Help_PromoterStrength('Ecol', Sequence), # noch zu ändern mit Help_PromoterStrength()?
        'Promoter': [Sequence],  # noch zu ändern (echt, random oder Fkt?)
        'ORF': ['ATGCCCCCCAAGTAA'], # noch zu ändern (echt, random oder Fkt?)
        'Fluxes': [9], # noch zu ändern mit Help_FluxCalculator()?
        'Expr2Flux': [4.5] # noch zu ändern mit Help_Expr2Flux()?
        }

        dfTF = pd.DataFrame(TF)
        newdf = newdf.append(dfTF, ignore_index=True)
    
        # add TF_regulated and RegType column:
        zeroarray = np.zeros(len(newdf),dtype=int)
        newdf = newdf.assign(TF_regulated=zeroarray) # später zu 0 oder jeweiliger TF-RctID ändern (random oder festgesetzt?)
        newdf = newdf.assign(RegType=zeroarray)
        
        TF_Indx = (newdf['RctID']==RegID) # access TF row
        newdf.loc[TF_Indx, 'RegType'] = 'Hill_Activator' # Metabolism Status: TF as activator
    
    # set Regulation of some Proteins:
    for ETarget in EnzymeTarget:
    # Future: fct recognizes what substrates are there, then makes TF_act or TF_rep
        myIndx = (newdf['RctID']==ETarget)
        newdf.loc[myIndx, 'TF_regulated'] = RegID

    return newdf


def make_DetectRegulatorPromoterMut(StrainWt:Strain, StrainMut:Strain, RctNewDF:DataFrame):
#     for obj in (StrainWt, StrainMut):
#         assert isinstance(obj, Strain), 'wrong strain type: must be Strain'
#     assert isinstance(RctNewDF, DataFrame), 'ReactNewDF of wrong type: input dataframe'
    GenesDF = StrainWt.genes_df
    # finding regulators in GenesDF
    myRegIndx = GenesDF[GenesDF['RegType'] != 0].index.values
    # finding index of regulators with changes in the promoter
    RegUpdatePromIndx = myRegIndx[RctNewDF.loc[myRegIndx, 'RctFlag'].values]
    # finding name of regulators with changes in the promoter
    RegUpdatePromName = RctNewDF.loc[RegUpdatePromIndx, 'RctID'].values
    # finding enzymes regulated by regulator
    EnzUpdateReguIndx = GenesDF['TF_regulated'].isin(RegUpdatePromName)
    
    return EnzUpdateReguIndx


def measure_EnzymeLevel(HostName:str, StrainWT:Strain, StrainMut:Strain):
    '''
    The function differences of expression levels of enzymes between two strains.
    '''
    RefGenDF = StrainWT.genes_df
    RefGenome = str(StrainWT.genome)
    MutGenome = str(StrainMut.genome)
    RefModel = StrainWT.model

    RctNewDF = Help_StrainCharacterizer(HostName, RefGenDF, RefGenome, MutGenome, RefModel)
    
    # identifying changes in regulator expression
    RegNewAr = make_DetectRegulatorPromoterMut(StrainWT, StrainMut, RctNewDF)

    # combining promoter changes enzyme + regulator
    AllNewAr = RctNewDF['RctFlag'].values | RegNewAr
    AllNewDF = deepcopy(RctNewDF)
    AllNewDF['RctFlag'] = AllNewAr  

    return AllNewDF


def calculate_StandardRegulatedExpression(WTdf):
    # finding the different regulator types and their equations which have their promoter changed
    RegTypes = [i for i in list(set(WTdf['TF_regulated'].values))]
    # removing the non regulated elements
    RegTypes.remove(0)
    print (RegTypes)

    PreProcFlux = deepcopy(WTdf['Expression'].values)
    
    for RegType in RegTypes:
        EnzIndx = WTdf['TF_regulated'] == RegType
        RegIndx = WTdf['RctID'] == RegType
        EquType = WTdf.loc[WTdf['RctID'] == RegType, 'RegType'].values
        MaxExpr = 2 * WTdf.loc[EnzIndx, 'Expression'].values # bei physiologischem cTF-Wert Flux= mtExpr
        
        cTF = WTdf.loc[RegIndx, 'Expression'].values # abhängig von geändertem TF-Promotor
        K = WTdf.loc[EnzIndx, 'Expression'].values # physiologischer Wert von c, Expression/Expr2Flux, beeinflusst Aktivität des TF, WT:nicht durch promotor geändert
        n = 1.95 # Kim, Harold D.; O'Shea, Erin K. (2008): A quantitative model of transcription factor–activated gene expression. DOI: 10.1038/nsmb.1500.  
#         print ('Eqn: ', RegType, MaxExpr, cTF, K)
        
        if EquType == 'Hill_Activator':
            Flux = MaxExpr / (1+(K/cTF)**n)
#             print(Flux)
   
        elif EquType == 'Hill_Repressor':
            Flux = MaxExpr * (1 - 1/ (1+(K/cTF)**n))

#         print(EnzIndx)
        PreProcFlux[EnzIndx] = Flux

    return PreProcFlux

    
def calculate_mutatedRegulatedExpression(WTdf, AllNewDF):
    # finding the different regulator types and their equations which have their promoter changed
    RegTypes = [i for i in list(set(WTdf.loc[AllNewDF['RctFlag'], 'TF_regulated'].values))]
    # removing the non regulated elements
    RegTypes.remove(0)

    for RegType in RegTypes:
        EnzIndx = WTdf['TF_regulated'] == RegType
        RegIndx = WTdf['RctID'] == RegType
        EquType = WTdf.loc[WTdf['RctID'] == RegType, 'RegType'].values
        MaxExpr = 2 * AllNewDF.loc[EnzIndx, 'RefExpr'].values # bei physiologischem cTF-Wert Flux= mtExpr

        cTF = AllNewDF.loc[RegIndx, 'NewExpr'].values # abhängig von geändertem TF-Promotor
        K = AllNewDF.loc[EnzIndx, 'RefExpr'].values # physiologischer Wert von c, Expression/Expr2Flux, beeinflusst Aktivität des TF, WT:nicht durch promotor geändert
        n = 1.95 # Kim, Harold D.; O'Shea, Erin K. (2008): A quantitative model of transcription factor–activated gene expression. DOI: 10.1038/nsmb.1500.  
#         print ('Eqn: ', AllNewDF)
        
        if EquType == 'Hill_Activator':
            corrExpr = MaxExpr / (1+(K/cTF)**n)
            
        elif EquType == 'Hill_Repressor':
            corrExpr = MaxExpr * (1 - 1/ (1+(K/cTF)**n))

            
        PreProcFlux = AllNewDF['NewExpr'].values
        PreProcFlux[EnzIndx] = corrExpr

    return PreProcFlux
        
def setboundary(WTdf, AllNewDF):
    '''
    Finding and setting the boundaries for cobrapy.
    '''

    # finding reactions for which the expression has changed
    OnlyEnzIndx = WTdf['RegType'] == 0
    RctNewDF = AllNewDF.loc[OnlyEnzIndx, ['RctFlag']] == True
    RctNew = np.arange((len(RctNewDF)))[np.ravel(RctNewDF.values)]
#     OnlyEnzIndx = WTdf[WTdf['RegType'] == 0].index.values
#     RctNew = AllNewDF[AllNewDF.loc[OnlyEnzIndx, 'RctFlag'] == True].index.values
    
    # For reactions with reduced and positive flux: reduce the upper limit,
    # For reactions with increased and positive flux: increase the lower limit
    # For reactions with negative flux the limits are exchanged.
    FluxPos = RctNew[tuple([AllNewDF.loc[RctNew, 'RefFlux']>0])]
    FluxNeg = RctNew[tuple([AllNewDF.loc[RctNew, 'RefFlux']<0])]
    # Finding increased and decreased fluxes
    FluxInc = RctNew[AllNewDF.loc[RctNew, 'NewFlux'].values / AllNewDF.loc[RctNew, 'RefFlux'].values>1]
    FluxDec = RctNew[AllNewDF.loc[RctNew, 'NewFlux'].values / AllNewDF.loc[RctNew, 'RefFlux'].values<1]

    # Comb.1: positive flux with increased expression -> increasing lower bound
    PosIncInd = np.intersect1d(FluxPos,FluxInc)
    # Comb.2: positive flux with decreased expression -> decreasing upper bound
    PosDecInd = np.intersect1d(FluxPos,FluxDec)
    # Comb.3: negative flux with increased expression -> decreasing lower bound
    NegIncInd = np.intersect1d(FluxNeg,FluxInc)
    # Comb.4: positive flux with increased expression -> increasing lower bound
    NegDecInd = np.intersect1d(FluxNeg,FluxDec)

    Expr_Change = {'increase': np.hstack([PosIncInd,NegIncInd]),'decrease': np.hstack([PosDecInd,NegDecInd])}
    Set_Boundary = {'lower': np.hstack([PosIncInd,NegDecInd]),'upper': np.hstack([PosDecInd,NegIncInd])}
    
#     Set_Boundary = {'lower': [0,2,3],'upper': []}
    return Set_Boundary

# Finding reactions to change. Three possibilities exits: 1. the promoter of the enzyme itself has changed, 2. the promoter of a regulator has changed, 3. the enzyme promoter and the regulator promoter have changed
# redefining Help_FluxCalculator in simulation/metabolism
def Help_FluxCalculator ( HostName:str, Strain:Strain, AllNewDF = None ) :
    '''
    Calculation of flux values.

    This method can work in 2 modes:
      [Strain only] : The metabolic model of the WT strain is used for calculation.
      [Strain + AllNewDF] : An additional step of resetting boundaries is done on the model
        before the fluxes are calculated.

    TODO: The "reset boundary" step is mutating the `StrainMut.model`, mutation might not be intended.
      Because of this, perhaps the other `Strain.model` gets mutated in the process.
    '''
#     from ..measurement.fluxes import measure_EnzymeLevel1

    # adding flux values
    # setup of flux boundaries. For the reference boundary changes are set to 'False',
    # for mutant strains, ractions with altered promoter sequence will change enzyme levels and boundaries must be changed accordingly, their variable is 'True'

    if AllNewDF is not None :
        print('resetting boundaries')
        # finding reactions for which the expression has changed, and finding the new level
        RefFlux = AllNewDF['RefFlux']
        NewFlux = AllNewDF['NewFlux']
        
        Set_Boundary = setboundary(Strain.genes_df, AllNewDF)
        # Defining the model with the two combinations of either
        # increasing lower bound (increased forward, decreased reverse reaction)
        # decreasing upper bound (decreased forward, increased reverse reaction)
        with Strain.model as myModel:
            # Comb.1: positive flux with increased expression -> increasing lower bound
            for Indx in Set_Boundary['lower']:
                myModel.reactions[Indx].lower_bound = AllNewDF.loc[Indx, 'NewFlux']
            # Comb.2: positive flux with decreased expression -> decreasing upper bound
            for Indx in Set_Boundary['upper']:
                myModel.reactions[Indx].upper_bound = AllNewDF.loc[Indx, 'NewFlux']

            Fluxes = myModel.optimize()

    else:
        Fluxes = Strain.model.optimize()


    return Fluxes.fluxes.values, Fluxes.objective_value


print('Functions defined.')


Functions defined.


## Wild type setup
The wild type is generated and transcription factor regulation is added manually.

In [3]:
wtHost = Ecol()
print('Wild type growth rate: {:.2f}'.format(wtHost.strain.objective))

# adding regulator TF to dataframe and genome of wt
WTdf = wtHost.strain.genes_df
WTdf = add_TranscriptionFactor(WTdf, 'TF1', ['PFK','PGI'])
WTdf = add_TranscriptionFactor(WTdf, 'TF2', ['PGL'])
def Update_DF(wtDF):
    # update for regulated expression
    corrExpr = calculate_StandardRegulatedExpression(wtDF)
    wtDF['Expression'] = corrExpr
    # update for Expression to Flux correlation, for all a linear factor
    myExpr2Flux = Help_Expr2Flux(wtDF['Fluxes'], corrExpr)
    wtDF['Expr2Flux'] = myExpr2Flux
    return wtDF
# update wildtype dataframe's Expression and Expr2Flux
WTdf = Update_DF(WTdf)
# putting the genes dataframe back to the wild type
wtHost.strain.genes_df = WTdf
# Generating new Genome
WTGenome_new = Help_GenomeGenerator(WTdf, 500, .6)
wtHost.strain.genome = Seq(WTGenome_new)


wtHost.strain.genes_df


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Wild type growth rate: 0.87


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


['TF2', 'TF1']


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


,RctID,Expression,Promoter,ORF,Fluxes,Expr2Flux,TF_regulated,RegType
0,PFK,2.579964,GCCCATTGAAGCAGCCGCTCAAGAGGCGTATTTTTGCACG,ATGGACCACCGGTAA,7.477382,2.898250,TF1,0
1,PFL,1.435000,GCCCATTGAGTAGAAATGATACCTTATTTAATTTTGCACG,ATGACCCTTCCTTAA,0.000000,0.000000,0,0
2,PGI,4.113402,GCCCATTGAGCTACATGTCGGGCAATTGTACAATTGCACG,ATGTTCTACTATTAA,4.860861,1.181713,TF1,0
3,PGK,0.799000,GCCCATTGATTGGCGAAGATGAAAAGTATATGGTTGCACG,ATGTGCCTCTACTGA,-16.023526,-20.054476,0,0
4,PGL,1.996444,GCCCATTGACGACACATCTGGCCGCTACTACTGTTGCACG,ATGCTTCCCCGGTAA,4.959985,2.484409,TF2,0
...,...,...,...,...,...,...,...,...
92,NH4t,1.332000,GCCCATTGATGTACGTGAATAACGAGACTAGACTTGCACG,ATGGATCTGATTTAA,4.765319,3.577567,0,0
93,O2t,1.644000,GCCCATTGAATCGCCTTGCTGCAATGCCTAATCTTGCACG,ATGCTAGAGTGGTAA,21.799493,13.260032,0,0
94,PDH,0.980000,GCCCATTGAGTCGTTGATAAGGAATCAATAGCGTTGCACG,ATGCCGGAACCCTAA,9.282533,9.471972,0,0
95,TF1,5.733000,GCCCATTGACAAGGCTCTCGCGGCCAGGTATAATTGCACG,ATGCCCCCCAAGTAA,9.000000,1.569859,0,Hill_Activator


## Constructing the mutant
Mutation targets are manually defined and also the mutations are pre-set. New expression is calculated based on the regulation type.

In [4]:
# Generating mutant
# We will actually define the mutations by hand and delete the automatically generated genome for now
myHost = deepcopy(wtHost)
# Mutants to be generated
Mutant_Targets = ['PFK','PGK','TF1']

for myMutant in Mutant_Targets:
#     Mutant_Bool = myHost.strain.genes_df['RctID']==myMutant
#     Mutant_Indx = tuple(np.arange(len(Mutant_Bool))[Mutant_Bool])
    MTdf = myHost.strain.genes_df.copy()
    Mutant_Indx = tuple(MTdf[MTdf['RctID'] == myMutant].index.values)
    # Finding the index for later printing
    WTIndx = str(wtHost.strain.genome).find(wtHost.strain.genes_df.loc[Mutant_Indx, 'Promoter'])
#     # Generating target promoter sequence, ATTGA is always there
    PromTar = MTdf.loc[Mutant_Indx, 'Promoter'].replace('ATTGA','CCCCC')
    # converting Biopython Seq class to string. This makes string replacements easier
    MutGenome = str(myHost.strain.genome)
    MutGenome = MutGenome.replace(MTdf.loc[Mutant_Indx, 'Promoter'], PromTar)
    myHost.strain.genome = Seq(MutGenome)
    MTdf.loc[Mutant_Indx, 'Promoter'] = PromTar
    myHost.strain.genes_df = MTdf.copy()
#     print('{}, Reference:\t{},\nMutated:\t{}\nwith index {}'.format(myMutant, wtHost.strain.genome[Mutant_Indx:Mutant_Indx+41], myHost.strain.genome[Mutant_Indx:Mutant_Indx+41], Mutant_Indx))
    print(myMutant, WTIndx)

# updating expression of regulated enzymes
AllNewDF = measure_EnzymeLevel('Ecol', wtHost.strain, myHost.strain)
CorrRegExpr = calculate_mutatedRegulatedExpression(WTdf, AllNewDF)
AllNewDF['NewExpr'] = CorrRegExpr
AllNewDF['NewFlux'] = AllNewDF['NewExpr']*WTdf['Expr2Flux']
AllNewDF

PFK 4
PGK 191
TF1 5720


,RctFlag,RctID,RefExpr,NewExpr,RefFlux,Expr2Flux,NewFlux
0,True,PFK,2.579964,3.211815,7.477382,2.898250,9.308643
1,False,PFL,1.435000,1.435000,0.000000,0.000000,0.000000
2,True,PGI,4.113402,3.282460,4.860861,1.181713,3.878925
3,True,PGK,0.799000,0.782000,-16.023526,-20.054476,-15.682600
4,False,PGL,1.996444,1.996444,4.959985,2.484409,4.959985
...,...,...,...,...,...,...,...
92,False,NH4t,1.332000,1.332000,4.765319,3.577567,4.765319
93,False,O2t,1.644000,1.644000,21.799493,13.260032,21.799493
94,False,PDH,0.980000,0.980000,9.282533,9.471972,9.282533
95,True,TF1,5.733000,3.334000,9.000000,1.569859,5.233909


## Calculation of mutant flux distribution
The updated expression activities are converted to reference flux values and according to the reference flux new boundaries are set in the GSMM. The wild type and mutant flux distributions are stored in a dictionary for visualization in Escher.

In [5]:
solution,_ = Help_FluxCalculator('Ecol', myHost.strain, AllNewDF)
RefFluxDict = WTdf.set_index('RctID')['Fluxes'].to_dict()
NewFluxDict = {a:b for a,b in zip(WTdf['RctID'].values, solution)}
AllFluxDict = [RefFluxDict, NewFluxDict]
# {WTdf['RctID'].tolist(): solution}

resetting boundaries


### Visualization of flux differences
Escher is used to visualize the flux differences on the central carbon metabolism map. Open the output-html file `Escher-Metabol-Map.html` by double click in the Jupyter Lab environment and `Trust` the file.

In [6]:
# builder = Builder()
# # Load an Escher map
# builder.map_name = 'iJO1366.Central metabolism'
# builder.model = wtHost.strain.model
# builder.reaction_data = AllFluxDict
# # Add some data for metabolites
# #builder.metabolite_data = solution.shadow_prices
# # Simplify the map by hiding some labels
# builder.hide_secondary_metabolites = True
# #builder.hide_all_labels = True
# builder.reaction_scale = [
#     { 'type': 'min', 'color': '#000000', 'size': 12 },
#     { 'type': 'median', 'color': '#ffffff', 'size': 20 },
#     { 'type': 'max', 'color': '#ff0000', 'size': 25 }
# ]
# builder.reaction_scale_preset = 'GaBuRd'

# # Make all the arrows three times as thick
# builder.reaction_scale = [
#     {k: v * 3 if k == 'size' else v for k, v in x.items()}
#     for x in builder.reaction_scale
# ]
# builder.save_html('Escher-Metabol-Map.html')

In [7]:
# create json file of AllFluxDict to 

import json

# save a single flux vector as JSON
flux_dictionary = AllFluxDict
filename = 'out'
with open(filename + '.json', 'w') as f:
    json.dump(flux_dictionary, f)

In [8]:
wtHost.strain.genes_df

,RctID,Expression,Promoter,ORF,Fluxes,Expr2Flux,TF_regulated,RegType
0,PFK,2.579964,GCCCATTGAAGCAGCCGCTCAAGAGGCGTATTTTTGCACG,ATGGACCACCGGTAA,7.477382,2.898250,TF1,0
1,PFL,1.435000,GCCCATTGAGTAGAAATGATACCTTATTTAATTTTGCACG,ATGACCCTTCCTTAA,0.000000,0.000000,0,0
2,PGI,4.113402,GCCCATTGAGCTACATGTCGGGCAATTGTACAATTGCACG,ATGTTCTACTATTAA,4.860861,1.181713,TF1,0
3,PGK,0.799000,GCCCATTGATTGGCGAAGATGAAAAGTATATGGTTGCACG,ATGTGCCTCTACTGA,-16.023526,-20.054476,0,0
4,PGL,1.996444,GCCCATTGACGACACATCTGGCCGCTACTACTGTTGCACG,ATGCTTCCCCGGTAA,4.959985,2.484409,TF2,0
...,...,...,...,...,...,...,...,...
92,NH4t,1.332000,GCCCATTGATGTACGTGAATAACGAGACTAGACTTGCACG,ATGGATCTGATTTAA,4.765319,3.577567,0,0
93,O2t,1.644000,GCCCATTGAATCGCCTTGCTGCAATGCCTAATCTTGCACG,ATGCTAGAGTGGTAA,21.799493,13.260032,0,0
94,PDH,0.980000,GCCCATTGAGTCGTTGATAAGGAATCAATAGCGTTGCACG,ATGCCGGAACCCTAA,9.282533,9.471972,0,0
95,TF1,5.733000,GCCCATTGACAAGGCTCTCGCGGCCAGGTATAATTGCACG,ATGCCCCCCAAGTAA,9.000000,1.569859,0,Hill_Activator


In [9]:
myHost.strain.genes_df

,RctID,Expression,Promoter,ORF,Fluxes,Expr2Flux,TF_regulated,RegType
0,PFK,2.579964,GCCCCCCCCAGCAGCCGCTCAAGAGGCGTATTTTTGCACG,ATGGACCACCGGTAA,7.477382,2.898250,TF1,0
1,PFL,1.435000,GCCCATTGAGTAGAAATGATACCTTATTTAATTTTGCACG,ATGACCCTTCCTTAA,0.000000,0.000000,0,0
2,PGI,4.113402,GCCCATTGAGCTACATGTCGGGCAATTGTACAATTGCACG,ATGTTCTACTATTAA,4.860861,1.181713,TF1,0
3,PGK,0.799000,GCCCCCCCCTTGGCGAAGATGAAAAGTATATGGTTGCACG,ATGTGCCTCTACTGA,-16.023526,-20.054476,0,0
4,PGL,1.996444,GCCCATTGACGACACATCTGGCCGCTACTACTGTTGCACG,ATGCTTCCCCGGTAA,4.959985,2.484409,TF2,0
...,...,...,...,...,...,...,...,...
92,NH4t,1.332000,GCCCATTGATGTACGTGAATAACGAGACTAGACTTGCACG,ATGGATCTGATTTAA,4.765319,3.577567,0,0
93,O2t,1.644000,GCCCATTGAATCGCCTTGCTGCAATGCCTAATCTTGCACG,ATGCTAGAGTGGTAA,21.799493,13.260032,0,0
94,PDH,0.980000,GCCCATTGAGTCGTTGATAAGGAATCAATAGCGTTGCACG,ATGCCGGAACCCTAA,9.282533,9.471972,0,0
95,TF1,5.733000,GCCCCCCCCCAAGGCTCTCGCGGCCAGGTATAATTGCACG,ATGCCCCCCAAGTAA,9.000000,1.569859,0,Hill_Activator


In [10]:
mylist = np.array(wtHost.strain.genes_df['Expression'])
mylist2 = np.array(myHost.strain.genes_df['Expression'])
print('Gene expression differences')
np.round(mylist2/mylist,2)

Gene expression differences


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [11]:
myl = np.array(wtHost.strain.genes_df['Expr2Flux'])
myl2 = np.array(myHost.strain.genes_df['Expr2Flux'])
# myHost.strain.genes_df
print('Exp2Flux diff')
np.round(myl2/myl,2)

Exp2Flux diff


<ipython-input-11-793ddc7a46d2>:5: RuntimeWarning: invalid value encountered in true_divide
  np.round(myl2/myl,2)


array([ 1., nan,  1.,  1.,  1., nan, nan,  1.,  1., nan, nan, nan,  1.,
        1.,  1.,  1., nan, nan, nan, nan,  1.,  1., nan,  1.,  1., nan,
        1.,  1.,  1.,  1., nan,  1., nan,  1., nan, nan,  1.,  1.,  1.,
       nan,  1.,  1.,  1., nan, nan, nan,  1., nan, nan, nan, nan,  1.,
       nan, nan,  1.,  1., nan, nan,  1.,  1.,  1., nan, nan,  1., nan,
       nan, nan, nan, nan,  1., nan,  1.,  1.,  1.,  1., nan,  1., nan,
       nan, nan,  1.,  1.,  1., nan, nan, nan, nan,  1., nan, nan,  1.,
       nan,  1.,  1.,  1.,  1.,  1.])

In [12]:
FluxDiff = myHost.strain.genes_df['Fluxes']/wtHost.strain.genes_df['Fluxes']
print('Flux reaction differences')
np.round(np.nan_to_num(FluxDiff, nan=1, posinf=1, neginf=1),2)

Flux reaction differences


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [18]:
pd.set_option('display.max_rows', None)
myHost.strain.genes_df['RctID'].to_csv


<bound method NDFrame.to_csv of 0                          PFK
1                          PFL
2                          PGI
3                          PGK
4                          PGL
5                        ACALD
6                       AKGt2r
7                          PGM
8                        PIt2r
9                       ALCD2x
10                      ACALDt
11                        ACKr
12                         PPC
13                      ACONTa
14                      ACONTb
15                        ATPM
16                        PPCK
17                       ACt2r
18                         PPS
19                        ADK1
20                       AKGDH
21                      ATPS4r
22                        PTAr
23                         PYK
24    BIOMASS_Ecoli_core_w_GAM
25                       PYRt2
26                        CO2t
27                         RPE
28                          CS
29                         RPI
30                    SUCCt2_2
31     